In [6]:
import pandas as pd
import numpy as np

# set of column names
headerNames = ["q1", "q2", "q3", "q4", "q5", "q6", "q7","q8", "q9", "q10", "gender", "age", "polAlign"]

# read csv-converted data from excel file
# current data is cleaned to exclude empty and/or meaningless columns
answerData = pd.read_csv('data/50data_answeronly.csv', header =0, names=headerNames)

# save number of entries
dataCount = len(answerData.index)

answerData

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign
0,③ 중립,① 매우 찬성,② 찬성,④ 반대,② 찬성,① 매우 찬성,③ 중립,② 찬성,③ 중립,② 찬성,남,40,2
1,① 매우 찬성,③ 중립,② 찬성,② 찬성,⑤ 매우 반대,④ 반대,② 찬성,④ 반대,③ 중립,① 매우 찬성,여,25,2
2,② 찬성,⑤ 매우 반대,③ 중립,② 찬성,① 매우 찬성,④ 반대,② 찬성,③ 중립,⑤ 매우 반대,② 찬성,남,31,3
3,⑤ 매우 반대,② 찬성,④ 반대,⑤ 매우 반대,① 매우 찬성,④ 반대,① 매우 찬성,① 매우 찬성,⑤ 매우 반대,③ 중립,남,51,4
4,③ 중립,③ 중립,③ 중립,③ 중립,② 찬성,③ 중립,③ 중립,③ 중립,③ 중립,③ 중립,여,38,3
5,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,④ 반대,② 찬성,③ 중립,여,47,3
6,① 매우 찬성,④ 반대,② 찬성,③ 중립,⑤ 매우 반대,① 매우 찬성,⑤ 매우 반대,② 찬성,① 매우 찬성,② 찬성,여,21,2
7,② 찬성,① 매우 찬성,② 찬성,② 찬성,④ 반대,③ 중립,③ 중립,② 찬성,③ 중립,① 매우 찬성,남,32,3
8,④ 반대,② 찬성,⑤ 매우 반대,④ 반대,④ 반대,④ 반대,④ 반대,① 매우 찬성,① 매우 찬성,① 매우 찬성,여,52,4
9,⑤ 매우 반대,③ 중립,⑤ 매우 반대,⑤ 매우 반대,① 매우 찬성,② 찬성,② 찬성,② 찬성,④ 반대,① 매우 찬성,남,56,5


In [9]:
# replace all redundant text to interger values

answerData.replace(["① 매우 찬성","② 찬성", "③ 중립", "④ 반대", "⑤ 매우 반대", "남", "여"], [1, 2, 3, 4, 5, "M", "F"], inplace = True)


# add new columns to denote how many times it has been assigned to pairs

answerData["same"] = [0]*dataCount
answerData["diff"] = [0]*dataCount

answerData

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,gender,age,polAlign,same,diff
0,3,1,2,4,2,1,3,2,3,2,M,40,2,0,0
1,1,3,2,2,5,4,2,4,3,1,F,25,2,0,0
2,2,5,3,2,1,4,2,3,5,2,M,31,3,0,0
3,5,2,4,5,1,4,1,1,5,3,M,51,4,0,0
4,3,3,3,3,2,3,3,3,3,3,F,38,3,0,0
5,2,4,2,4,2,4,2,4,2,3,F,47,3,0,0
6,1,4,2,3,5,1,5,2,1,2,F,21,2,0,0
7,2,1,2,2,4,3,3,2,3,1,M,32,3,0,0
8,4,2,5,4,4,4,4,1,1,1,F,52,4,0,0
9,5,3,5,5,1,2,2,2,4,1,M,56,5,0,0


In [7]:
# create a new dataframe to save values
# this will be very time and cost extensive, might be viable to change this part
# will be durable for several hundred calculations though...?

diffMatrix = pd.DataFrame(np.nan, index=range(dataCount), columns=range(dataCount), dtype='float')
diffMatrix

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
for i in range(dataCount):
    for j in range(dataCount):
        if (i == j):
            continue
        for k in range(10):
            # iterates through 10 for the number of questions in the dataframe
            tempVal=0;
            tempVal += answerData.iloc[i][k]-answerData.iloc[j][k]
            diffMatrix.iloc[i][j] = abs(tempVal)

diffMatrix

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
1,1.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
3,1.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,...,0.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0
4,1.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,...,0.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0
5,1.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,...,0.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
7,1.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,0.0
8,1.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,0.0
9,1.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,0.0
